In [55]:

import requests as rq
import pandas as pd
import plotly.express as px
import numpy as np

In [2]:
# !pip install requests
# !pip install plotly
# !pip install pandas

# !pip install openpyxl # ler excel
# !pip install matplotlib

# ValueError: Mime type rendering requires nbformat>=4.2.0 but it is not installed
# !pip install --upgrade nbformat
# !pip install nbformat 
# !pip install jupyter


[notice] A new release of pip is available: 24.0 -> 24.3.1
[notice] To update, run: C:\Users\amtra\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


In [3]:
# -*- coding: utf-8 -*-
"""
Editor Spyder

Este é um arquivo de script temporário.
"""

import requests as rq
import pandas as pd

# =============================================================================
# capital
# https://www.ssp.sp.gov.br/v1/OcorrenciasMensais/RecuperaDadosMensaisAgrupados?
# ano=2023&grupoDelito=6&tipoGrupo=REGI%C3%83O&idGrupo=1
# =============================================================================

# =============================================================================
# sao paulo
# https://www.ssp.sp.gov.br/v1/OcorrenciasMensais/RecuperaDadosMensaisAgrupados?ano=2023&grupoDelito=6&tipoGrupo=MUNIC%C3%8DPIO&idGrupo=565
# 
# =============================================================================

#grupoDelito=6&
url = 'https://www.ssp.sp.gov.br/v1/OcorrenciasMensais/'+\
    'RecuperaDadosMensaisAgrupados?ano=2023&grupoDelito=6&'+\
    'tipoGrupo=ESTADO&idGrupo=0'

headers = {'authority':'www.ssp.sp.gov.br',
'method':'GET',
'path':'/v1/OcorrenciasMensais/RecuperaDadosMensaisAgrupados?ano=2023&grupoDelito=6&tipoGrupo=ESTADO&idGrupo=0',
'scheme':'https',
'accept':'application/json',
'accept-encoding':'gzip, deflate, br, zstd',
'accept-language':'pt-BR,pt;q=0.9,en-US;q=0.8,en;q=0.7,de-DE;q=0.6,de;q=0.5',
'authorization':'',
'cache-control':'no-cache',
'dnt':'1',
'pragma':'no-cache',
'priority':'u=1, i',
'referer':'https://www.ssp.sp.gov.br/estatistica/dados-mensais',
'sec-ch-ua':'"Google Chrome";v="131", "Chromium";v="131", "Not_A Brand";v="24"',
'sec-ch-ua-mobile':'?0',
'sec-ch-ua-platform':'"Windows"',
'sec-fetch-dest':'empty',
'sec-fetch-mode':'cors',
'sec-fetch-site':'same-origin',
'user-agent':'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/131.0.0.0 Safari/537.36',
'x-requestid':'22616d98-d167-4ab8-a72c-0a58bb5ceba7'}

listRemKey = ['ano','delito','idDelito','idDistrito','publicado',
              'idOcorrenciaMensal','total']
dictMes = {'janeiro':1,
'fevereiro':2,
'marco':3,
'abril':4,
'maio':5,
'junho':6,
'julho':7,
'agosto':8,
'setembro':9,
'outubro':10,
'novembro':11,
'dezembro':12}

# r = rq.get(url,headers=headers)
dfOcorr = pd.DataFrame()
for y in list(range(2001,2024)) + [0]:
    nomey = str(y)
    url = 'https://www.ssp.sp.gov.br/v1/OcorrenciasMensais/'+\
        'RecuperaDadosMensaisAgrupados?ano='+nomey+'&grupoDelito=6&'+\
        'tipoGrupo=ESTADO&idGrupo=0'
    
    r = rq.get(url)
    

    if r.status_code == 200:
        j = r.json()
        nomeAno = j['data'][0]['ano']
        dfcat = pd.DataFrame()
        listDados = j['data'][0]['listaDados']
        delito = listDados[1]
        for delito in listDados:
            nomeDelito = delito['delito']['delito']
            delitoCopy = delito.copy()
            for k in listRemKey:
                delitoCopy.pop(k)
            df = pd.DataFrame([delitoCopy]).T
            df.columns=[nomeDelito]
            df.index.name = 'MES'
            listDatas = df.reset_index()['MES'].apply(lambda x : pd.Timestamp(nomeAno,dictMes[x],1)).to_list()
            df['DATA'] = listDatas 
            df
            
            df = df.set_index('DATA',drop=True)
            
            dfcat = pd.concat([dfcat,df],axis=1)
    else:
        print('error em '+nomey)
    
    dfOcorr = pd.concat([dfOcorr,dfcat],axis=0)
    
dfOcorr.to_csv('dados_ssp_mensal.csv',index=True) 



In [112]:
dfOcorr.columns


Index(['HOMICÍDIO DOLOSO (2)', 'Nº DE VÍTIMAS EM HOMICÍDIO DOLOSO (3)',
       'HOMICÍDIO DOLOSO POR ACIDENTE DE TRÂNSITO',
       'Nº DE VÍTIMAS EM HOMICÍDIO DOLOSO POR ACIDENTE DE TRÂNSITO',
       'HOMICÍDIO CULPOSO POR ACIDENTE DE TRÂNSITO',
       'HOMICÍDIO CULPOSO OUTROS', 'TENTATIVA DE HOMICÍDIO',
       'LESÃO CORPORAL SEGUIDA DE MORTE', 'LESÃO CORPORAL DOLOSA',
       'LESÃO CORPORAL CULPOSA POR ACIDENTE DE TRÂNSITO',
       'LESÃO CORPORAL CULPOSA - OUTRAS', 'LATROCÍNIO',
       'Nº DE VÍTIMAS EM LATROCÍNIO', 'TOTAL DE ESTUPRO (4)',
       'TOTAL DE ROUBO - OUTROS (1)', 'ROUBO DE VEÍCULO', 'ROUBO A BANCO',
       'ROUBO DE CARGA', 'FURTO - OUTROS', 'FURTO DE VEÍCULO', 'ESTUPRO',
       'ROUBO - OUTROS', 'ESTUPRO DE VULNERÁVEL'],
      dtype='object')

In [146]:
dfOcorrSazonal = dfOcorr.iloc[:-2,:].copy()
dfOcorrSazonal = dfOcorrSazonal.loc[dfOcorrSazonal.index > pd.Timestamp(2012,1,1),:]
dfRollMean = dfOcorrSazonal.rolling(window=12,min_periods=1).mean()
dfOcorrSazonal = dfOcorrSazonal - dfRollMean
dfOcorrSazonal = dfOcorrSazonal.reset_index()
dfOcorrSazonal['MES'] = dfOcorrSazonal['DATA'].apply(lambda x : x.month)

In [151]:
listCol = list(dfOcorr.columns)
col = listCol[0]
for col in listCol:
    dfTemp0 = dfOcorr.loc[:,[col]].copy()
    dfTemp0['type']='variable'
    dfTemp0 = dfTemp0.iloc[:-2,:]
    dfTemp1 = dfRollMean.loc[:,[col]].copy()
    dfTemp1['type']='12m_rol_win'
    dfTemp1 = dfTemp1.iloc[:-2,:]
    dftemp = pd.concat([dfTemp0,dfTemp1],axis=0)

    #fig = px.line(dfOcorr.iloc[:-2,:],y=col,title='Série Temporal')
    
    fig = px.line(dftemp,y=col,color='type',title='Série Temporal')

    fig.show()
    fig = px.box(dfOcorrSazonal,x='MES',y=col,title='Sazonalidade Mensal')
    fig.show()

In [ ]:

# =============================================================================
# 
# 
# https://www.ssp.sp.gov.br/assets/estatistica/transparencia/baseDados/celularesSub/CelularesSubtraidos_2024.xlsx
# 
# 
# https://www.ssp.sp.gov.br/assets/estatistica/transparencia/baseDados/veiculosSub/VeiculosSubtraidos_2024.xlsx
# 
# 
# https://www.ssp.sp.gov.br/assets/estatistica/transparencia/spDados/SPDadosCriminais_2024.xlsx
# 
# 
# https://www.ssp.sp.gov.br/assets/estatistica/transparencia/spDados/DadosProdutividade_2024.xlsx
# 
# 
# =============================================================================

